In [24]:
import numpy as np
import nltk
import numpy as np
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import svm
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import string

train_file = "data/1600000/training.csv"
test_file = "data/1600000/test.csv"
lemmatizer = WordNetLemmatizer()
word_list = set(w.lower() for w in words.words())
stop_words = set(stopwords.words('english'))
punc = string.punctuation


In [26]:
print stop_words

set([u'all', u'just', u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u'do', u'them', u'his', u'very', u'they', u'not', u'during', u'now', u'him', u'nor', u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u'where', u'few', u'because', u'doing', u'some', u'hasn', u'are', u'our', u'ourselves', u'out', u'what', u'for', u'while', u're', u'does', u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u'were', u'here', u'shouldn', u'hers', u'by', u'on', u'about', u'couldn', u'of', u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u'mightn', u'wasn', u'your', u'from', u'her', u'their', u'aren', u'there', u'been', u'whom', u'too', u'wouldn', u'themselves', u'weren', u'was', u'until', u'more', u'himself', u'that', u'but', u'don', u'with', u'than', u'those', u'he', u'me', u'myself', u'ma', u'these', u'up', u'will', u'below', u'ain

In [29]:
lemmatizer.lemmatize("horrible")

'horrible'

In [78]:
# print stop_words
with open("data/1600000/punct.csv",'wb') as f:
    for word in list(punc):
        print word
        f.write(word)
        f.write('\n')
    f.close()

!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
:
;
<
=
>
?
@
[
\
]
^
_
`
{
|
}
~


In [ ]:
s = "0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
s
s[0]

In [2]:
import csv
label = []
time = []
text = []

test_label = []
test_text = []
with open(train_file,'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for line in spamreader:
        label.append(line[0])
        text.append(line[5])
    csvfile.close()

print len(label),len(text)

with open(test_file,'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for line in spamreader:
        if line[0] != "2":
            test_label.append(line[0])
            test_text.append(line[5])
    csvfile.close()

1600000 1600000


In [14]:
len(test_label)
len(test_text)

359

In [3]:
# try:

all_text = []
n = 0
for line in text:
    sentence = []
    s = line.split()
    for word in s:
        if len(word)>2 and word not in stop_words:      
            word = re.sub("[^A-Za-z]","",word)
            if word:
                try:
                    word = lemmatizer.lemmatize(word.lower())
                except:
                    word = word.lower()
                sentence.append(word)
    all_text.append(' '.join(sentence))
    n += 1

print len(text)
print len(all_text)

1600000
1600000


In [4]:
all_test_text = []
n = 0
for line in test_text:
    sentence = []
    s = line.split()
    for word in s:
        if len(word)>2 and word not in stop_words:      
            word = re.sub("[^A-Za-z]","",word)
            if word:
                try:
                    word = lemmatizer.lemmatize(word.lower())
                except:
                    word = word.lower()
                sentence.append(word)
    all_test_text.append(' '.join(sentence))
    n += 1

print len(all_test_text)

359


In [5]:
train_input = open("data/1600000/training.pkl",'wb')
test_input = open("data/1600000/test.pkl",'wb')
pickle.dump(all_text,train_input)
pickle.dump(all_text,train_input)

In [6]:
vectorizer = CountVectorizer(min_df=1,binary = True)
text_model = vectorizer.fit_transform(all_text)
test_model = vectorizer.transform(all_test_text)

In [7]:
text_model.shape,test_model.shape

((1600000, 777666), (359, 777666))

In [9]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [8]:
from sklearn import linear_model
lg_p2 = linear_model.LogisticRegression(penalty='l2',C=1e5,solver = 'newton-cg')
lg_p1 = linear_model.LogisticRegression(penalty='l1',C=1e5,solver = 'liblinear')
lg_p2.fit(text_model,label)
lg_p1.fit(text_model,label)
predict1 = lg_p1.predict(test_model)
predict2 = lg_p2.predict(test_model)

/Users/apple/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


NameError: name 'precision_score' is not defined

In [34]:
path = open("data/1600000/lg1.pkl",'wb')
pickle.dump(lg_p1,path)
path = open("data/1600000/lg2.pkl",'wb')
pickle.dump(lg_p2,path)

In [32]:
predict1 = map(int,list(predict1))
predict2 = map(int,list(predict2))
test_label = map(int,list(test_label))

[0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 

In [33]:

lg1_pre = precision_score(test_label,predict1)
lg2_pre = precision_score(test_label,predict2)
lg1_recall = recall_score(test_label,predict1)
lg2_recall = recall_score(test_label,predict2)
lg1_f1 = f1_score(test_label,predict1)
lg2_f1 = f1_score(test_label,predict2)


print lg1_pre,lg1_recall,lg1_f1
print lg2_pre,lg2_recall,lg2_f1

0.767441860465 0.725274725275 0.745762711864
0.775862068966 0.741758241758 0.758426966292


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=0)
scores_nb = cross_val_score(dt,text_model,label,cv= 1,scoring='f1_macro')
print np.mean(scores_nb)

In [35]:
from sklearn.svm import LinearSVC
nsvc = LinearSVC(random_state=0)
nsvc.fit(text_model,label)


TypeError: 'LinearSVC' object is not callable

In [66]:
predict = nsvc.predict(test_model)
predict = map(int,list(predict))
for n in range(len(test_label)):
    if test_label[n] == 4:
        test_label[n] = 1
    if predict[n] == 4:
        predict[n] =1
svm_pre = precision_score(test_label,predict)
svm_recall = recall_score(test_label,predict)
svm_f1 = f1_score(test_label,predict)
print svm_pre,svm_recall,svm_f1


0.814207650273 0.818681318681 0.816438356164


In [54]:
path = open("data/1600000/svm.pkl",'wb')
pickle.dump(nsvc,path)

In [64]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=3, random_state=0)
rf.fit(text_model,label)
predict = rf.predict(test_model)
predict = map(int,list(predict))
for n in range(len(test_label)):
    if test_label[n] == 4:
        test_label[n] = 1
    if predict[n] == 4:
        predict[n] =1
rf_pre = precision_score(test_label,predict)
rf_recall = recall_score(test_label,predict)
rf_f1 = f1_score(test_label,predict)
print rf_pre,rf_recall,rf_f1

0.514450867052 0.978021978022 0.674242424242


In [62]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(text_model,label)
predict = nb.predict(test_model)
predict = map(int,list(predict))
for n in range(len(test_label)):
    if test_label[n] == 4:
        test_label[n] = 1
    if predict[n] == 4:
        predict[n] =1
mnb_pre = precision_score(test_label,predict)
mnb_recall = recall_score(test_label,predict)
mnb_f1 = f1_score(test_label,predict)
print mnb_pre,mnb_recall,mnb_f1

0.830508474576 0.807692307692 0.818941504178


In [65]:
path = open("data/1600000/randomforest.pkl",'wb')
pickle.dump(rf,path)
path = open("data/1600000/multinomialNB.pkl",'wb')
pickle.dump(nb,path)